## 高斯过程（GP）是一种通用的监督学习方法，旨在解决回归和概率分类问题。
### 高斯过程的优点是：
* 预测内插了观察结果（至少对于正则核）。
* 预测是概率（Gaussian），所以人们可以计算经验置信区间，并根据那些应该在某些兴趣区域改编（在线拟合，自适应）预测的那些来确定。
* 通用性：可以指定不同的内核。提供通用内核，但也可以指定自定义内核。
### 高斯过程的缺点包括：
* 它们不稀疏，即他们使用整个样本/特征信息来执行预测。
* 它们在高维空间中失去效率 - 即当功能数量超过几十个时。

## 高斯过程回归（GPR）
该GaussianProcessRegressor工具高斯过程（GP）回归的目的。为此，需要指定GP的先前。假设先前的均值为常数，零（normalize_y=False或）训练数据的平均值（for normalize_y=True）。先前的协方差是通过传递内核对象来指定的。在通过最大化基于传递的对数边缘似然（LML）的GaussianProcessRegressor拟合期间，优化内核的超参数optimizer。由于LML可能具有多个局部最优值，所以优化器可以通过指定重复启动n_restarts_optimizer。第一次运行始终从内核的初始超参数值开始执行; 从已经从允许值的范围中随机选择的超参数值进行后续运行。<BR>
可以通过参数传递目标中的噪声级别alpha，全局为标量或每数据点。请注意，适度的噪声水平也可以有助于处理拟合期间的数字问题，因为它被有效地实现为Tikhonov正则化，即通过将其添加到核心矩阵的对角线。明确指定噪声水平的替代方法是将WhiteKernel组件包含在内核中，这可以从数据中估计全局噪声电平（见下面的示例）。
实现基于[RW2006]的算法2.1。除了标准scikit学习估计器的API之外，GaussianProcessRegressor：
* 允许预测，无需事先拟合（基于GP之前）
* 提供了一种额外的方法sample_y(X)，其评估在给定输入处从GPR（先验或后验）绘制的样本
* 公开了一种方法log_marginal_likelihood(theta)，可以在外部使用其他方式选择超参数，例如通过马尔科夫链蒙特卡罗链。

## 高斯过程的内核
内核（在GP的背景下也称为协方差函数）是确定GP前后形状的GP的关键组成部分。他们通过定义两个数据点的“相似性”，并结合相似的数据点应该具有相似的目标值的假设，对所学习的函数进行编码。可以区分两类内核：固定内核只取决于两个数据点的距离，而不是依赖于它们的绝对值，因此它们对于输入空间中的转换是不变的，而非固定的内核也取决于数据点的具体值。固定内核可以进一步细分为各向同性和各向异性内核，其中各向同性内核也不会在输入空间中旋转。更多细节，
## 高斯过程内核API 
a的主要用法Kernel是计算GP在数据点之间的协方差。为此，__call__可以调用内核的方法。该方法可以用于计算2d阵列X中所有数据点对的“自动协方差”，或二维阵列X的数据点与二维阵列Y中的数据点的所有组合的“互协方差”。以下身份对于所有内核k都是正确的（除了WhiteKernel）： k(X) == K(X, Y=X)
如果仅使用自动协方差的对角线，diag() 则可以调用内核的方法，该方法比等效调用的计算效率更高__call__：np.diag(k(X, X)) == k.diag(X)
内核通过超参数向量参数化。这些超参数可以例如控制内核的长度尺度或周期性（见下文）。内核的自协方差对于所有的内核支持计算解析梯度通过设置 eval_gradient=True在__call__方法。高斯过程（回归和分类器）都使用这种梯度来计算对数边缘似然的梯度，后者又用于通过梯度上升来确定最大化对数边缘似然的值。对于每个超参数，在创建内核实例时需要指定初始值和边界。可以通过theta内核对象的属性获取和设置 当前值。此外，bounds超参数的范围可以通过内核的属性来访问。请注意，这两个属性（theta和bounds）都会返回内部使用值的日志转换值，因为这些值通常更适合基于梯度的优化。每个超级参数的规范Hyperparameter以相应内核中的实例的形式存储 。请注意，使用名称为“x”的超参数的内核必须具有self.x和self.x_bounds属性。每个超级参数的规范以相应内核中的实例的形式存储。请注意，使用名称为“x”的超参数的内核必须具有self.x和self.x_bounds属性。每个超级参数的规范以相应内核中的实例的形式存储。请注意，使用名称为“x”的超参数的内核必须具有self.x和self.x_bounds属性。
所有内核的抽象基类是Kernel。内核实现类似的接口Estimator，提供方法get_params()， set_params()和clone()。这也允许通过诸如Pipeline或者之类的元估计来设置内核值GridSearch。请注意，由于内核的嵌套结构（通过应用内核操作符，请参见下文），内核参数的名称可能会变得相对复杂。一般来说，对于二进制内核运算符，左操作数的k1__ 参数与右操作数的参数前缀k2__。一个额外的便利方法是clone_with_theta(theta)返回克隆版本的内核，但是超参数设置为theta。

In [1]:
import numpy as np 
from sklearn import gaussian_process
def f(x):
    return x*np.sin(x)

In [2]:
X = np.atleast_2d([1., 3., 5., 6., 7., 8.]).T
y = f(X).ravel()
x = np.atleast_2d(np.linspace(0, 10, 1000)).T
gp = gaussian_process.GaussianProcess(theta0=1e-2, thetaL=1e-4, thetaU=1e-1)
gp.fit(X, y) 

/home/wenshao/anaconda3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class GaussianProcess is deprecated; GaussianProcess was deprecated in version 0.18 and will be removed in 0.20. Use the GaussianProcessRegressor instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/wenshao/anaconda3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function l1_cross_distances is deprecated; l1_cross_distances was deprecated in version 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/home/wenshao/anaconda3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function constant is deprecated; The function constant of regression_models is deprecated in version 0.19.1 and will be removed in 0.22.
  warnings.warn(msg, category=DeprecationWarning)
/home/wenshao/anaconda3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function squared

GaussianProcess(beta0=None,
        corr=<function squared_exponential at 0x7f649f21f268>,
        normalize=True, nugget=array(2.220446049250313e-15),
        optimizer='fmin_cobyla', random_start=1,
        random_state=<mtrand.RandomState object at 0x7f64bc07db88>,
        regr=<function constant at 0x7f649f218d90>, storage_mode='full',
        theta0=array([[ 0.01]]), thetaL=array([[ 0.0001]]),
        thetaU=array([[ 0.1]]), verbose=False)

In [4]:
y_pred, sigma2_pred = gp.predict(x, eval_MSE=True)
y_pred
sigma2_pred

/home/wenshao/anaconda3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function constant is deprecated; The function constant of regression_models is deprecated in version 0.19.1 and will be removed in 0.22.
  warnings.warn(msg, category=DeprecationWarning)
/home/wenshao/anaconda3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function squared_exponential is deprecated; The function squared_exponential of correlation_models is deprecated in version 0.19.1 and will be removed in 0.22.
  warnings.warn(msg, category=DeprecationWarning)


array([  4.33238499e+00,   4.17638003e+00,   4.02497320e+00,
         3.87805595e+00,   3.73552171e+00,   3.59726587e+00,
         3.46318575e+00,   3.33318062e+00,   3.20715160e+00,
         3.08500167e+00,   2.96663567e+00,   2.85196023e+00,
         2.74088378e+00,   2.63331649e+00,   2.52917028e+00,
         2.42835879e+00,   2.33079734e+00,   2.23640291e+00,
         2.14509412e+00,   2.05679121e+00,   1.97141602e+00,
         1.88889196e+00,   1.80914397e+00,   1.73209855e+00,
         1.65768365e+00,   1.58582876e+00,   1.51646478e+00,
         1.44952408e+00,   1.38494041e+00,   1.32264895e+00,
         1.26258623e+00,   1.20469013e+00,   1.14889987e+00,
         1.09515596e+00,   1.04340023e+00,   9.93575749e-01,
         9.45626844e-01,   8.99499068e-01,   8.55139185e-01,
         8.12495147e-01,   7.71516072e-01,   7.32152232e-01,
         6.94355025e-01,   6.58076966e-01,   6.23271661e-01,
         5.89893791e-01,   5.57899096e-01,   5.27244353e-01,
         4.97887360e-01,